In [27]:
import torchvision
import torch.nn as nn
import torch.nn.functional as nnf
import torch
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms.functional import to_tensor, normalize
import time
import matplotlib.pyplot as plt
from IPython.display import clear_output
from torchvision import transforms, models
from torchvision.datasets import Food101
from torchvision import datasets, transforms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
print(torch.cuda.get_device_name(0))

def test_acc(net: nn.Module, test_loader: DataLoader):

  net.to(device)
  net.eval()
  
  total = 0
  correct = 0

  for images, labels in test_loader:
    images, labels = images.to(device), labels.to(device)
    total += labels.size(0)

    outputs = net(images)
    _, predicted = torch.max(outputs, 1)
    correct += (predicted == labels).sum().item()

  return correct / total * 100

def train_fn(epochs: int, train_loader: DataLoader,
             net: nn.Module, loss_fn: nn.Module, optimizer: optim.Optimizer, train_dataset_length):

  # losses = []
  # accuracies = []

  net.to(device)

  for e in range(epochs):
    net.train()
    running_loss = 0.0

    for images, labels in train_loader:
      images, labels = images.to(device), labels.type(torch.LongTensor).to(device)

      optimizer.zero_grad()
      outputs = net(images)
      loss = loss_fn(outputs, labels)
      loss.backward()
      optimizer.step()

      running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / train_dataset_length

    print(f"Loss: {epoch_loss}")

    # acc = test_acc(net, test_loader)
    # print(f"Acuratetea la finalul epocii {e + 1} este {acc:.2f}%")

    # losses.append(epoch_loss)
    # accuracies.append(acc)

    # torch.save(net.state_dict(), f'resnet50_multiplefclayers_adamw_epoch{e + 1}.pkl')

  # loss_graph(losses)
  # accuracy_graph(accuracies)

def loss_graph(losses):
    plt.plot(losses)

    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.show()

def accuracy_graph(accuracies):
    plt.plot(accuracies)

    plt.xlabel('epochs')
    plt.ylabel('accuracy')
    plt.show()

class CustomClassifier(nn.Module):
    def __init__(self):
        super(CustomClassifier, self).__init__()

        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((1, 1)),
            nn.Flatten(),
            nn.Linear(2048, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            # nn.Dropout(0.25),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            # nn.Dropout(0.1),
            nn.Linear(128, 101)
        )

    def forward(self, x):
        return self.classifier(x)

transform = transforms.Compose([
    # transforms.RandomHorizontalFlip(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225])
])

cuda
NVIDIA GeForce RTX 4060 Laptop GPU


In [14]:
# optimizer = optim.SGD(resnet.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0001)
epochs = 1
accuracies = []

for split_number in range(1, 6):

    train_dir = f"C:/Users/Vlad Talpiga.VLR_PROJAMZ/OneDrive - Valrom Industrie SRL/Desktop/IAVA/Proiect/FoodClassifier/cross_validation/dataset/split{split_number}/train"
    val_dir = f"C:/Users/Vlad Talpiga.VLR_PROJAMZ/OneDrive - Valrom Industrie SRL/Desktop/IAVA/Proiect/FoodClassifier/cross_validation/dataset/split{split_number}/validation"
 
    train_dataset = datasets.ImageFolder(train_dir, transform=transform)
    val_dataset = datasets.ImageFolder(val_dir, transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

    resnet = models.resnet50(pretrained=True)
    resnet = nn.Sequential(*list(resnet.children())[:-1])

    # Freeze all layers except the final classification layer
    for param in resnet.parameters():
        param.requires_grad = False

    resnet = nn.Sequential(
        resnet,
        CustomClassifier()
    )

    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.Adam(resnet.parameters(), lr=0.001)

    start = time.time()
    print(f'Training split {split_number}')

    train_fn(epochs, train_loader, resnet, loss_fn, optimizer, len(train_dataset))

    acc = test_acc(resnet, val_loader)

    print(f'Accuracy: {acc:.2f}%')
    accuracies.append(acc)

    end = time.time()
    print(f'Duration of training: {(end - start) / 60}')

Training split 1
Loss-ul la finalul epocii 1: 2.2068452828473384
Accuracy: 51.84%
Duration of training: 9.886932826042175
Training split 2
Loss-ul la finalul epocii 1: 2.1992753799362936
Accuracy: 51.64%
Duration of training: 9.019914611180623
Training split 3
Loss-ul la finalul epocii 1: 2.2046103348118242
Accuracy: 52.77%
Duration of training: 9.183107888698578
Training split 4
Loss-ul la finalul epocii 1: 2.2165430443829828
Accuracy: 53.41%
Duration of training: 8.866597191492717
Training split 5
Loss-ul la finalul epocii 1: 2.2024331460612836
Accuracy: 52.06%
Duration of training: 9.638064348697663


In [16]:
import statistics as st

print(accuracies)
print(st.mean(accuracies))

[51.836633663366335, 51.64356435643565, 52.76732673267327, 53.410891089108915, 52.06435643564357]
52.34455445544555


In [21]:
# SGD

configs = [(0.9, 0.001), (0.9, 0.0001), (0.9, 0.00001), (0.95, 0.001), (0.95, 0.0001), (0.95, 0.00001), (0.99, 0.001), (0.99, 0.0001), (0.99, 0.00001)]
mean_accuracies = []
i = 1

for momentum, weight_decay in configs:

    print(f'Config {i}:')
    i += 1
    accuracies = []

    for split_number in range(1, 6):

        train_dir = f"C:/Users/Vlad Talpiga.VLR_PROJAMZ/OneDrive - Valrom Industrie SRL/Desktop/IAVA/Proiect/FoodClassifier/cross_validation/dataset/split{split_number}/train"
        val_dir = f"C:/Users/Vlad Talpiga.VLR_PROJAMZ/OneDrive - Valrom Industrie SRL/Desktop/IAVA/Proiect/FoodClassifier/cross_validation/dataset/split{split_number}/validation"
    
        train_dataset = datasets.ImageFolder(train_dir, transform=transform)
        val_dataset = datasets.ImageFolder(val_dir, transform=transform)

        train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

        resnet = models.resnet50(pretrained=True)
        resnet = nn.Sequential(*list(resnet.children())[:-1])

        # Freeze all layers except the final classification layer
        for param in resnet.parameters():
            param.requires_grad = False

        resnet = nn.Sequential(
            resnet,
            CustomClassifier()
        )

        loss_fn = nn.CrossEntropyLoss()
        optimizer = optim.SGD(resnet.parameters(), lr=0.001, momentum=momentum, weight_decay=weight_decay)

        start = time.time()
        print(f'Training split {split_number}')

        train_fn(1, train_loader, resnet, loss_fn, optimizer, len(train_dataset))

        acc = test_acc(resnet, val_loader)

        print(f'Accuracy: {acc:.2f}%')
        accuracies.append(acc)

        end = time.time()
        print(f'Duration of training: {(end - start) / 60}')

    mean_accuracies.append(st.mean(accuracies))
    
    print(accuracies)
    print(st.mean(accuracies))

print(mean_accuracies)

Config 1:


c:\Users\Vlad Talpiga.VLR_PROJAMZ\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Vlad Talpiga.VLR_PROJAMZ\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Training split 1
Loss: 2.887284066511853
Accuracy: 49.94%
Duration of training: 8.890704516569773
Training split 2
Loss: 2.8999206365453136
Accuracy: 49.34%
Duration of training: 8.301789673169454
Training split 3
Loss: 2.885908913706789
Accuracy: 49.84%
Duration of training: 8.290831764539083
Training split 4
Loss: 2.898863299908024
Accuracy: 50.05%
Duration of training: 15.497810331980387
Training split 5
Loss: 2.8987160547653046
Accuracy: 48.55%
Duration of training: 18.473181653022767
[49.94059405940594, 49.34158415841584, 49.84158415841584, 50.054455445544555, 48.554455445544555]
49.54653465346534
Config 2:
Training split 1
Loss: 2.8880823403537863
Accuracy: 49.90%
Duration of training: 18.513389825820923
Training split 2
Loss: 2.8969056493929117
Accuracy: 49.13%
Duration of training: 18.377429684003193
Training split 3
Loss: 2.8913166430445
Accuracy: 48.92%
Duration of training: 18.516337180137633
Training split 4
Loss: 2.883935251424808
Accuracy: 50.03%
Duration of training: 18.

In [22]:
# Adam

accuracies = []

for split_number in range(1, 6):

    train_dir = f"C:/Users/Vlad Talpiga.VLR_PROJAMZ/OneDrive - Valrom Industrie SRL/Desktop/IAVA/Proiect/FoodClassifier/cross_validation/dataset/split{split_number}/train"
    val_dir = f"C:/Users/Vlad Talpiga.VLR_PROJAMZ/OneDrive - Valrom Industrie SRL/Desktop/IAVA/Proiect/FoodClassifier/cross_validation/dataset/split{split_number}/validation"
 
    train_dataset = datasets.ImageFolder(train_dir, transform=transform)
    val_dataset = datasets.ImageFolder(val_dir, transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

    resnet = models.resnet50(pretrained=True)
    resnet = nn.Sequential(*list(resnet.children())[:-1])

    # Freeze all layers except the final classification layer
    for param in resnet.parameters():
        param.requires_grad = False

    resnet = nn.Sequential(
        resnet,
        CustomClassifier()
    )

    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.Adam(resnet.parameters(), lr=0.0009)

    start = time.time()
    print(f'Training split {split_number}')

    train_fn(1, train_loader, resnet, loss_fn, optimizer, len(train_dataset))

    acc = test_acc(resnet, val_loader)

    print(f'Accuracy: {acc:.2f}%')
    accuracies.append(acc)

    end = time.time()
    print(f'Duration of training: {(end - start) / 60}')

print(accuracies)
print(st.mean(accuracies))

Training split 1
Loss: 2.2097909264517304
Accuracy: 52.74%
Duration of training: 8.832646040121714
Training split 2
Loss: 2.21071876558927
Accuracy: 51.55%
Duration of training: 9.191163639227549
Training split 3
Loss: 2.209246954870696
Accuracy: 52.24%
Duration of training: 8.57719300587972
Training split 4
Loss: 2.2220643899936485
Accuracy: 53.00%
Duration of training: 8.775090579191843
Training split 5
Loss: 2.2116764879698803
Accuracy: 52.04%
Duration of training: 8.692610148588816
[52.73762376237624, 51.554455445544555, 52.23762376237624, 52.9950495049505, 52.03960396039604]
52.312871287128715


In [28]:
# AdamW

configs = [(0.9, 0.999, 0.01), (0.9, 0.999, 0.001), (0.9, 0.999, 0.0001), (0.95, 0.999, 0.01), (0.95, 0.999, 0.001), (0.95, 0.999, 0.0001), (0.85, 0.999, 0.01), (0.85, 0.999, 0.001), (0.85, 0.999, 0.0001)]
mean_accuracies = []
i = 1

for b1, b2, weight_decay in configs:

    print(f'Config {i}:')
    i += 1
    accuracies = []

    for split_number in range(1, 6):

        train_dir = f"C:/Users/Vlad Talpiga.VLR_PROJAMZ/OneDrive - Valrom Industrie SRL/Desktop/IAVA/Proiect/FoodClassifier/cross_validation/dataset/split{split_number}/train"
        val_dir = f"C:/Users/Vlad Talpiga.VLR_PROJAMZ/OneDrive - Valrom Industrie SRL/Desktop/IAVA/Proiect/FoodClassifier/cross_validation/dataset/split{split_number}/validation"
    
        train_dataset = datasets.ImageFolder(train_dir, transform=transform)
        val_dataset = datasets.ImageFolder(val_dir, transform=transform)

        train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

        resnet = models.resnet50(pretrained=True)
        resnet = nn.Sequential(*list(resnet.children())[:-1])

        # Freeze all layers except the final classification layer
        for param in resnet.parameters():
            param.requires_grad = False

        resnet = nn.Sequential(
            resnet,
            CustomClassifier()
        )

        loss_fn = nn.CrossEntropyLoss()
        optimizer = optim.AdamW(resnet.parameters(), lr=0.001, betas=(b1, b2), weight_decay=weight_decay)

        start = time.time()
        print(f'Training split {split_number}')

        train_fn(1, train_loader, resnet, loss_fn, optimizer, len(train_dataset))

        acc = test_acc(resnet, val_loader)

        print(f'Accuracy: {acc:.2f}%')
        accuracies.append(acc)

        end = time.time()
        print(f'Duration of training: {(end - start) / 60}')

    mean_accuracies.append(st.mean(accuracies))
    
    print(accuracies)
    print(st.mean(accuracies))

print(mean_accuracies)

Config 1:
Training split 1
Loss: 2.2109824182963607
Accuracy: 52.51%
Duration of training: 10.534675590197246
Training split 2
Loss: 2.204332356216884
Accuracy: 51.34%
Duration of training: 12.625886758168539
Training split 3
Loss: 2.2044516959992966
Accuracy: 51.84%
Duration of training: 12.583011571566264
Training split 4
Loss: 2.223164396946973
Accuracy: 52.83%
Duration of training: 12.023633607228597
Training split 5
Loss: 2.200249844211163
Accuracy: 51.22%
Duration of training: 12.417280972003937
[52.50990099009901, 51.336633663366335, 51.836633663366335, 52.831683168316836, 51.21782178217822]
51.94653465346535
Config 2:
Training split 1
Loss: 2.210957428724459
Accuracy: 53.21%
Duration of training: 11.757775886853535
Training split 2
Loss: 2.211444401127277
Accuracy: 52.38%
Duration of training: 11.935789803663889
Training split 3
Loss: 2.198007081428377
Accuracy: 52.43%
Duration of training: 11.678800185521444
Training split 4
Loss: 2.2131138474870435
Accuracy: 52.44%
Duration o